# Understanding the execution flow from `LightningQubit`

In [ ]:
from typing import Iterable, Union
import quimb.tensor as qtn
import pennylane as qml
from pennylane.wires import Wires
import numpy as np

## Approach 1: choosing a tape

In [ ]:
theta = 0.1
phi = 0.2

O1 = qml.Identity(wires=[0])
O2 = qml.PauliZ(wires=[1])

ops = [
    qml.DoubleExcitation(phi, wires=[12, 11, 0, 9]),
    qml.PauliX(wires=[0]),
    qml.RX(theta, wires=[0]),
    qml.qml.CSWAP(wires=[7, 0, 5]),
    qml.RX(phi, wires=[1]),
    qml.CNOT(wires=[3, 4]),
    qml.DoubleExcitation(phi, wires=[1, 2, 3, 4]),
    qml.qml.CSWAP(wires=[0, 1, 2]),
    qml.Hadamard(wires=[4]),
    qml.qml.CSWAP(wires=[2, 3, 4]),
    qml.CNOT(wires=[2, 4]),
    qml.DoubleExcitation(phi, wires=[0, 1, 3, 4]),
]

meas = [
    qml.expval(O2),
    qml.expval(O2),
    qml.var(qml.PauliY(wires=[2])),
    qml.expval(O1),
    qml.var(qml.PauliY(wires=[4])),
]


tape = qml.tape.QuantumScript(ops=ops, measurements=meas)

In [ ]:
tape.wires

In [ ]:
tape.operations

In [ ]:
tape.observables

In [ ]:
tape.measurements

In [ ]:
tape.wires

In [ ]:
dev = qml.device("lightning.qubit", wires=tape.wires)

dev.execute(circuits=tape)

## Learning from `LightningQubit` using Python API

In [ ]:
from pennylane_lightning.lightning_qubit import LightningQubit

In [ ]:
obj = LightningQubit(wires=tape.wires)

In [ ]:
obj.tracker

In [ ]:
program, config = obj.preprocess()

In [ ]:
program[0]

In [ ]:
obj.execute(tape)

## Testing `LightningTensor` 

In [ ]:
wires = tape.wires
backend = "quimb"
method = "mps"
c_dtype = np.complex128
max_bond_dim = None
cutoff = 0.0

In [ ]:
from pennylane_lightning.lightning_tensor import LightningTensor

In [ ]:
from pennylane_lightning.lightning_tensor import LightningTensor

@qml.qnode(LightningTensor(contract="auto-mps"))
def circuit():
    qml.X(0)
    return qml.expval(qml.Z(0))

circuit()

In [ ]:
from pennylane_lightning.lightning_tensor import LightningTensor



@qml.qnode(LightningTensor(max_bond_dim=100,  wires=qml.wires.Wires(range(2))))
def circuit():
    qml.Z(0)
    return qml.expval(qml.Z(0))

circuit()

In [ ]:
qml.wires.Wires(range(2))

In [ ]:
obj = LightningTensor(
    wires=wires,
    backend=backend,
    method=method,
    c_dtype=c_dtype,
    max_bond_dim=max_bond_dim,
    cutoff=cutoff,
)

In [ ]:
obj._interface.state

In [ ]:
# obj._interface._circuitMPS.get_psi_reverse_lightcone(3)

In [ ]:
type(obj._setup_execution_config())

In [ ]:
program, config = obj.preprocess()

In [ ]:
program[0]

In [ ]:
config

In [ ]:
# program[0]

In [ ]:
# config

In [ ]:
# config.device_options

In [ ]:
obj.execute(circuits=tape)

In [ ]:
obj._interface._circuitMPS.gates

In [ ]:
obj._interface._circuitMPS.psi.tags

In [ ]:
obj._interface.state

In [ ]:
obj._interface._circuitMPS.__dict__

In [ ]:
obj._interface._circuitMPS

In [ ]:
obj._interface._circuitMPS.psi.max_bond()

In [ ]:
obj._interface._circuitMPS.psi

In [ ]:
# obj._interface.draw_state()

In [ ]:
obj._interface._circuitMPS.psi.to_dense()

In [ ]:
# obj.compute_derivatives(tape)

In [ ]:
# obj._interface.state_to_array()

In [ ]:
obj._interface._circuitMPS._psi

In [ ]:
progbar=True,

# Playing with `quimb`

In [ ]:
wires = Wires(range(7))

init_state = qtn.MPS_computational_state(
    "0" * len(wires), tags=[str(l) for l in wires.labels]
)

qc = qtn.CircuitMPS(psi0=init_state)

op = qml.CNOT(wires=[0, 1])

In [ ]:
qc.psi.draw(color=[f"I{q}" for q in range(len(wires))], show_tags=False, show_inds=True)

In [ ]:
init_state.gate_

In [ ]:
qc.amplitude("0111111")

In [ ]:
init_state

In [ ]:
qc.ket_site_ind

In [ ]:
qc.__dict__

In [ ]:
qc.gates

In [ ]:
gate = qtn.circuit.parse_to_gate(op.matrix(), tuple(op.wires))

In [ ]:
gate

In [ ]:
qtn.MPS_computational_state("0" * len(wires), tags=[str(l) for l in wires.labels])

In [ ]:
qtn.MPS_zero_state(bond_dim=6, L=9)

In [ ]:
qtn.MPS_rand_state(L=9, bond_dim=6)

# Question for the guy

In [ ]:
import pennylane as qml
import quimb.tensor as qtn
import numpy as np

# case 1: single qubit gate (works fine)
op1 = qml.PauliX(wires=[0])
qtn.Circuit(N=1).apply_gate(
    op1.matrix().astype(np.complex128), *op1.wires, parametrize=None
)

# case 2: two qubit gate (works fine)
op2 = qml.CNOT(wires=[0, 1])
qtn.Circuit(N=2).apply_gate(
    op2.matrix().astype(np.complex128), *op2.wires, parametrize=None
)

# case 3: three qubit gate (fails)
op3 = qml.Toffoli(wires=[0, 1, 2])
# qtn.CircuitMPS(N=3).apply_gate(op3.matrix(), *op3.wires, parametrize=None)

# case 4: four qubit gate (fails)
op4 = qml.DoubleExcitation(0.1, wires=[0, 1, 2, 3])
# qtn.Circuit(N=4).apply_gate(op4.matrix().astype(np.complex128), *op4.wires, parametrize=None)

In [ ]:
ops = [
    qml.PauliX(wires=[0]),
    qml.RX(theta, wires=[0]),
    qml.CSWAP(wires=[7, 0, 5]),
    qml.RX(phi, wires=[1]),
    qml.CNOT(wires=[3, 4]),
    qml.DoubleExcitation(phi, wires=[1, 2, 3, 4]),
    qml.CZ(wires=[4, 5]),
    qml.Hadamard(wires=[4]),
    qml.CCZ(wires=[0, 1, 2]),
    qml.CSWAP(wires=[2, 3, 4]),
    qml.QFT(wires=[0, 1, 2]),
    qml.CNOT(wires=[2, 4]),
    qml.Toffoli(wires=[0, 1, 2]),
    qml.DoubleExcitation(phi, wires=[0, 1, 3, 4]),
]

meas = [
    qml.expval(O2),
    qml.expval(O2),
    qml.expval(O1),
    qml.var(qml.PauliY(wires=[2])),
    qml.expval(O1),
    qml.var(qml.PauliY(wires=[4])),
]


tape = qml.tape.QuantumScript(ops=ops, measurements=meas)

In [ ]:
obj = LightningQubit(wires=tape.wires)

obj.execute(tape)

In [ ]:
qml.Rot(0.2, -0.4, 0.3, wires=[2]).matrix()

In [ ]:
obj = LightningTensor(
    wires=wires,
    backend=backend,
    method=method,
    c_dtype=c_dtype,
    max_bond_dim=max_bond_dim,
    cutoff=cutoff,
)


obj.execute(tape)

In [ ]:
import pennylane as qml

# Define the operations
ops = [
    qml.Hadamard(wires=[0]),
    qml.CRX(0.5, wires=[1, 2]),
    qml.CNOT(wires=[0, 1]),
    qml.CRX(0.3, wires=[2, 3]),
    qml.PauliX(wires=[0]),
    qml.RX(theta, wires=[0]),
    qml.CSWAP(wires=[7, 0, 5]),
    qml.RX(phi, wires=[1]),
    qml.CNOT(wires=[3, 4]),
    qml.DoubleExcitation(phi, wires=[1, 2, 3, 4]),
    qml.CZ(wires=[4, 5]),
    qml.Hadamard(wires=[4]),
    qml.CCZ(wires=[0, 1, 2]),
    qml.CSWAP(wires=[2, 3, 4]),
    qml.QFT(wires=[0, 1, 2]),
    qml.CNOT(wires=[2, 4]),
    qml.Toffoli(wires=[0, 1, 2]),
    qml.DoubleExcitation(phi, wires=[0, 1, 3, 4]),
    qml.CSWAP(wires=[1, 2, 3]),
    qml.RX(-0.8, wires=[0]),
    qml.CZ(wires=[0, 2]),
    qml.RY(0.4, wires=[1]),
    qml.CNOT(wires=[1, 3]),
    qml.CCZ(wires=[0, 1, 2]),
    qml.Hadamard(wires=[3])
]

# Define the measurement
meas = [
    qml.expval(qml.PauliX(wires=[0])),
    qml.var(qml.PauliY(wires=[1])),
    qml.expval(qml.PauliZ(wires=[2])),
    qml.expval(qml.PauliX(wires=[3]))
]

# Create the QuantumScript tape
tape = qml.tape.QuantumScript(ops=ops, measurements=meas)

In [ ]:
obj = LightningQubit(
    wires=tape.wires,
    c_dtype=c_dtype,
)


obj.execute(tape)

In [ ]:
obj = LightningTensor(
    wires=tape.wires,
    backend=backend,
    method=method,
    c_dtype=c_dtype,
    max_bond_dim=max_bond_dim,
    cutoff=cutoff,
)


obj.execute(tape)

In [ ]:
import pennylane as qml

# Define the operations
ops = [
    qml.Rot(-0.3, 0.7, -0.4, wires=[0]),
    qml.RY(0.8, wires=[1]),
    qml.CNOT(wires=[1, 2]),
    qml.CRX(0.2, wires=[0, 2]),
    qml.PauliY(wires=[1]),
    qml.CZ(wires=[0, 3]),
    qml.Hadamard(wires=[2]),
    qml.CSWAP(wires=[1, 2, 3]),
    qml.Rot(0.1, 0.5, 0.3, wires=[0]),
    qml.CRX(-0.6, wires=[2, 3]),
    qml.RX(0.4, wires=[1]),
    qml.CNOT(wires=[0, 1]),
    qml.Toffoli(wires=[0, 1, 3])
]

# Define the measurement
meas = [
    qml.var(qml.PauliX(wires=[0])),
    qml.expval(qml.PauliY(wires=[1])),
    qml.expval(qml.PauliZ(wires=[2])),
    qml.var(qml.PauliX(wires=[3]))
]

# Create the QuantumScript tape
tape = qml.tape.QuantumScript(ops=ops, measurements=meas)


In [ ]:
obj = LightningTensor(
    wires=wires,
    backend=backend,
    method=method,
    c_dtype=c_dtype,
    max_bond_dim=max_bond_dim,
    cutoff=cutoff,
)


obj.execute(tape)

In [ ]:
import pennylane as qml

# Define the operations
ops = [
    qml.Rot(0.1, -0.5, 0.6, wires=[0]),
    qml.RX(-0.2, wires=[1]),
    qml.CSWAP(wires=[0, 1, 2]),
    qml.RY(0.3, wires=[2]),
    qml.CNOT(wires=[1, 3]),
    qml.PauliX(wires=[2]),
    qml.CRX(0.4, wires=[0, 3]),
    qml.Hadamard(wires=[1]),
    qml.CCZ(wires=[0, 1, 3]),
    qml.CRX(0.7, wires=[2, 3]),
    qml.CZ(wires=[0, 1]),
    qml.Toffoli(wires=[0, 2, 3])
]

# Define the measurement
meas = [
    qml.expval(qml.PauliZ(wires=[0])),
    qml.var(qml.PauliY(wires=[1])),
    qml.expval(qml.PauliZ(wires=[2])),
    qml.var(qml.PauliX(wires=[3]))
]

# Create the QuantumScript tape
tape = qml.tape.QuantumScript(ops=ops, measurements=meas)


In [ ]:
obj = LightningTensor(
    wires=wires,
    backend=backend,
    method=method,
    c_dtype=c_dtype,
    max_bond_dim=max_bond_dim,
    cutoff=cutoff,
)


obj.execute(tape)

In [ ]:
def from_op_to_tensor(op) -> qtn.Tensor:
    """Returns the Quimb tensor corresponding to a PennyLane operator."""
    wires = tuple(op.wires)
    bra_inds = []
    for _, i in enumerate(wires):
        bra_inds.append(f"b{i}")
    bra_inds = tuple(bra_inds)
    ket_inds = []
    for _, i in enumerate(wires):
        ket_inds.append(f"k{i}")
    ket_inds = tuple(ket_inds)
    array = op.matrix().astype(np.complex128)
    return qtn.Tensor(array.reshape([2] * int(np.log2(array.size))), inds=bra_inds + ket_inds)


In [ ]:
op = qml.DoubleExcitation(0.1, wires=[0, 1, 2, 3])

from_op_to_tensor(op)

In [ ]:
import numpy as np
import pennylane as qml


def pure_ising(n):

    """
    Creates an Ising Hamiltonian \sum_{i<j} Z_iZ_j on n qubits
    """

    coeffs = []
    ops = []

    for i in range(n):
        for j in range(i):
            coeffs.append(1)
            ops.append(qml.PauliZ(i) @ qml.PauliZ(j))

    return qml.dot(coeffs, ops)


def transverse_field(n):
    """
    Creates a transverse field operator \sum_i X_i on n qubits
    """

    coeffs = []
    ops = []

    for i in range(n):
            coeffs.append(1)
            ops.append(qml.PauliX(i))

    return qml.dot(coeffs, ops)


def magnetization(n):
    """
    Creates a total magnetization operator \sum_i Z_i on n qubits
    """

    coeffs = []
    ops = []

    for i in range(n):
            coeffs.append(1)
            ops.append(qml.PauliZ(i))

    return qml.dot(coeffs, ops)


def transverse_ising(J, h, n):
    """
       Creates a transverse field Ising Hamiltonian H = J \sum_{i<j} Z_iZ_j + h \sum_i X_i
       :param J: coupling strength
       :param h: transverse field strength
       :param n: number of qubits
       :return: Hamiltonian
       """

    return J * pure_ising(n) + h * transverse_field(n)


J = 12.1  # coupling field
h = -2.5  # transverse field
nr_qubits = 7  # number of qubits

Hamiltonian = transverse_ising(J, h, nr_qubits)

print(f"This is the Hamiltonian: {Hamiltonian}")
print(magnetization(nr_qubits), type(magnetization(nr_qubits)))

dev = qml.device('lightning.qubit', wires=nr_qubits)

#dev = LightningTensor(
#    wires=nr_qubits,
#)

@qml.qnode(dev)
def time_evolution(t):

    qml.TrotterProduct(Hamiltonian, t, n=2, order=4)
    
    M = magnetization(nr_qubits)

    return qml.expval(M)

dt = 0.05  # time step in evolution

In [ ]:
#time_evolution(dt)